# Visualization

This tutorial will show some of the available interactive visualizations that are available in (or compatible with) SpikeInterface.

In [ ]:
from pathlib import Path


import spikeinterface as si
import spikeinterface.preprocessing as spre
import spikeinterface.postprocessing as spost
import spikeinterface.curation as scur
import spikeinterface.widgets as sw
import spikeinterface.qualitymetrics

In [ ]:
si.set_global_job_kwargs(n_jobs=-1, progress_bar=True)

Let's first load some data:

In [ ]:
base_folder = Path("../../SpikeInterface Dataset Tutorial/")

# base_folder = Path("/home/samuel/DataSpikeSorting/Edinburgh_SI_tutorials/")


In [ ]:
postprocessing_folder = base_folder / "dataset_postprocessing"

recording = si.load_extractor(postprocessing_folder / "recording")
sorting = si.load_extractor(postprocessing_folder / "sorting_mysterious")

In [ ]:
recording

In [ ]:
sorting

## sortingview


[sortingview](https://github.com/magland/sortingview) is a Python package developed by Jeremy Magland that visualizes ephys data in the cloud.

It uses the [figurl](https://github.com/flatironinstitute/figurl/blob/main/README.md) technology to push the visualization data to the cloud and produce self-contained URL link that points will allow you to visualize and interact with the data from any computer connected to the internet.

Pretty cool, right??

To get started, you just need to run this command, click on the link, and login with your GitHub account:

In [ ]:
!kachery-cloud-init

There are a few plot_* functions that support `sortingview` as backend, but the two most useful ones are the:

- `plot_traces()`
- `plot_sorting_summary()`

### plot_traces()

In [ ]:
# let's do some preprocessing
recording = spre.depth_order(recording)
recording_hp = spre.highpass_filter(recording)
recording_cmr = spre.common_reference(recording_hp)


recording_layers = dict(
    raw=recording,
    highpass=recording_hp,
    cmr=recording_cmr
)

w = sw.plot_traces(
    recording_layers,
    mode="map",
    order_channel_by_depth=True,
    time_range=[0, 0.2], 
    figlabel="SpikeInterface tutorial: plot_traces",
    clim=(-50, 50),
    backend="sortingview"
)

# plot_sorting_summary

For this plot, we need an analyzer and some extensions:

In [ ]:
analyzer = si.create_sorting_analyzer(sorting, recording)

In [ ]:
required_extensions = [
    "random_spikes",
    "waveforms",
    "templates",
    "noise_levels",
    "unit_locations",
    "template_similarity",
    "spike_amplitudes",
    "correlograms"
]
analyzer.compute(required_extensions)
analyzer

We can add any property to the units table:

In [ ]:
num_spikes = sorting.count_num_spikes_per_unit()
unit_amplitudes = si.get_template_extremum_amplitude(analyzer)

analyzer.sorting.set_property("num_spikes", list(num_spikes.values()))
analyzer.sorting.set_property("amplitude", list(unit_amplitudes.values()))

In [ ]:
w = sw.plot_sorting_summary(
    analyzer,
    unit_table_properties=["num_spikes", "amplitude"], 
    curation=True, 
    label_choices=["noise", "MUA", "SUA"],
    backend="sortingview"
)

After curating, we can apply the curation either from the downloaded JSON file or with the URI:

In [ ]:
curation_json = "sorting-curation.json"

sorting_curated_from_json = scur.apply_sortingview_curation(sorting, uri_or_json=curation_json)
sorting_curated_from_json

In [ ]:
uri = "sha1://090dcb14f4a57781862be59a20ed5732b8341cdd"
sorting_curated_from_uri = scur.apply_sortingview_curation(sorting, uri_or_json=uri)
sorting_curated_from_uri

## GUIs

In [ ]:
analyzer_saved = analyzer.save_as(
    folder=base_folder / "analyzer_for_visualization",
)

### plot_traces with `ephyviewer`

In [ ]:
%gui qt
sw.plot_traces(recording, backend="ephyviewer")

### plot_sorting_summary with `spikeinterface-gui`

In [ ]:
analyzer_saved.compute("principal_components", n_components=3, mode="by_channel_global")
analyzer_saved.compute("quality_metrics", metric_names=["snr"])
analyzer_saved

In [ ]:
%gui qt
sw.plot_sorting_summary(analyzer_saved, backend="spikeinterface_gui")